In [9]:
import pandas as pd
import os
import re
from datetime import datetime

def read_files(path: str) -> pd.DataFrame:
    arr_dfs = []
    for file in os.listdir(path):
        if file.endswith('.csv'):
            
            pattern = r"\d{4}-\d{2}"
            match = re.search(pattern, file)
            if match:
                date_str = match.group(0)
                year, month = map(int, date_str.split('-'))
                ref_date = datetime(year, month, 1)
                
                df = pd.read_csv(path + '/' + file)
                df['ref_date'] = ref_date
                arr_dfs.append(df)

            else:
                print("No date found: ", file)
    
    if len(arr_dfs):
        return pd.concat(arr_dfs)
    else:
        return None
    
def credit_card_pipeline(dataframe: pd.DataFrame) -> pd.DataFrame:

    from_to_category = {'payment': 'fatura'}

    category_from_title = {
        'Buser':'transporte', 'Mp *Buser':'transporte',
        'Assai Atacadista':'supermercado', 'Gympass Gympassbr':'saúde',
        'Wellhub Gympass Br Gym':'saúde', 'Openai *Chatgpt Subscr':'serviços'}
    
    installment_pattern = r"\b(?:0|[1-9]|[1-9][0-9]|100)/(?:0|[1-9]|[1-9][0-9]|100)\b"

    # format date
    dataframe['date'] = pd.to_datetime(dataframe['date'], format='%Y-%m-%d')

    # remove installment number of title
    dataframe['fmt_title'] = dataframe['title'].apply(lambda x: re.sub(installment_pattern, "", x).strip())

    # flag rows where the title contains installment number
    dataframe['is_installment'] = dataframe['title'].str.contains(installment_pattern)

    # create new category column
    dataframe['new_category'] = dataframe['category'].copy()
    dataframe['new_category'] = dataframe['new_category'].replace(from_to_category)
    for title, category in category_from_title.items():
        dataframe.loc[dataframe['title'] == title, 'new_category'] = category


    return dataframe

Load Data

In [14]:
data_path = 'D:/Workspace/data/personal_finances/nubank/{0}'

# Credit Card Data
df_credit_card_raw = read_files(data_path.format('/credit_card'))
df_credit_card = credit_card_pipeline(df_credit_card_raw)
df_credit_card.to_csv(data_path.format('/credit_card_database.csv'), index=False, encoding='utf-8')

In [12]:
df_credit_card.dtypes

date              datetime64[ns]
category                  object
title                     object
amount                   float64
ref_date          datetime64[us]
fmt_title                 object
is_installment              bool
new_category              object
dtype: object

In [13]:
df_credit_card.head()

,date,category,title,amount,ref_date,fmt_title,is_installment,new_category
0,2024-06-03,supermercado,Horti Fruti Ortencia,60.82,2024-06-01,Horti Fruti Ortencia,False,supermercado
1,2024-06-01,restaurante,Pani Di Grano,41.20,2024-06-01,Pani Di Grano,False,restaurante
2,2024-05-31,restaurante,Pinheiro Food,53.98,2024-06-01,Pinheiro Food,False,restaurante
3,2024-05-31,supermercado,Pag*Minimercado,21.75,2024-06-01,Pag*Minimercado,False,supermercado
4,2024-05-31,casa,Assai Atacadista,707.77,2024-06-01,Assai Atacadista,False,supermercado


In [3]:
print(df_credit_card['date'].min(), df_credit_card['date'].max())

display(df_credit_card.head())
display(df_credit_card.query('is_installment == True').head())

2024-05-05 00:00:00 2024-08-03 00:00:00


,date,category,title,amount,fmt_title,is_installment,new_category
0,2024-06-03,supermercado,Horti Fruti Ortencia,60.82,Horti Fruti Ortencia,False,supermercado
1,2024-06-01,restaurante,Pani Di Grano,41.20,Pani Di Grano,False,restaurante
2,2024-05-31,restaurante,Pinheiro Food,53.98,Pinheiro Food,False,restaurante
3,2024-05-31,supermercado,Pag*Minimercado,21.75,Pag*Minimercado,False,supermercado
4,2024-05-31,casa,Assai Atacadista,707.77,Assai Atacadista,False,supermercado


,date,category,title,amount,fmt_title,is_installment,new_category
45,2024-05-05,outros,Mercadolivre*Mercadol 4/5,91.92,Mercadolivre*Mercadol,True,outros
46,2024-05-05,casa,Americanas *Madeira M 7/8,83.14,Americanas *Madeira M,True,casa
47,2024-05-05,educação,Htm*Alexvargasfno 7/12,28.94,Htm*Alexvargasfno,True,educação
48,2024-05-05,serviços,Vindi *Kinvo 4/12,19.90,Vindi *Kinvo,True,serviços
49,2024-05-05,eletrônicos,Pag*Daianelimademedei 2/2,58.50,Pag*Daianelimademedei,True,eletrônicos


In [4]:
category_from_title = ['Buser','Mp *Buser','Assai Atacadista','Gympass Gympassbr','Wellhub Gympass Br Gym','Openai *Chatgpt Subscr']
for c in category_from_title:
    display(df_credit_card.loc[df_credit_card['title'] == c][['title','category','new_category']])

,title,category,new_category
8,Buser,charge,transporte


,title,category,new_category
30,Mp *Buser,serviços,transporte
42,Mp *Buser,serviços,transporte


,title,category,new_category
4,Assai Atacadista,casa,supermercado
16,Assai Atacadista,casa,supermercado


,title,category,new_category
11,Gympass Gympassbr,outros,saúde
12,Gympass Gympassbr,outros,saúde


,title,category,new_category
3,Wellhub Gympass Br Gym,outros,saúde


,title,category,new_category
2,Openai *Chatgpt Subscr,eletrônicos,serviços
